In [1]:
import pandas as pd
import matplotlib.pyplot as plt
from sklearn.model_selection import train_test_split
from sklearn.ensemble import RandomForestClassifier
from sklearn import metrics
%matplotlib inline

traindata = pd.read_csv("train_values.csv",sep=',')
traindata.columns[traindata.isna().any()]
data=traindata
data.drop(['building_id'], axis=1, inplace=True)
trainlabels = pd.read_csv("train_labels.csv",sep=',')
trainout=trainlabels
trainout.drop(['building_id'], axis=1, inplace=True)
testvalues = pd.read_csv("test_values.csv",sep=',')
testdata=testvalues
testdata.drop(['building_id'], axis=1, inplace=True)

In [2]:
cleanup_letters = {"land_surface_condition": {"n": 1, "o": 2, "t":3},
                "foundation_type": {"h": 1, "i": 2, "r": 3, "u": 4, "w": 5},               
                "roof_type": {"n": 1, "q": 2, "x": 3},
                "ground_floor_type": {"f": 1, "m": 2, "v": 3, "x": 4, "z": 5},
                "other_floor_type": {"j": 1, "q": 2, "s": 3, "x": 4},
                "position": {"j": 1, "o": 2, "s": 3, "t": 4},
                "plan_configuration": {"a": 1, "c": 2, "d": 3, "f": 4, "m": 5, "n": 6, "o": 7, "q": 8, "s": 9, "u":10},
                "legal_ownership_status": {"a": 1, "r": 2, "v": 3, "w": 4}             
               }
data.replace(cleanup_letters, inplace=True)
testdata.replace(cleanup_letters, inplace=True)
a = list(data)
del a[-1]
x = data[a].values
y = trainout['damage_grade'].values

In [3]:
clf = RandomForestClassifier(n_estimators=50,max_features=8)
clf.fit(x, y)

RandomForestClassifier(bootstrap=True, class_weight=None, criterion='gini',
            max_depth=None, max_features=8, max_leaf_nodes=None,
            min_impurity_decrease=0.0, min_impurity_split=None,
            min_samples_leaf=1, min_samples_split=2,
            min_weight_fraction_leaf=0.0, n_estimators=50, n_jobs=None,
            oob_score=False, random_state=None, verbose=0,
            warm_start=False)

In [4]:
test=testdata[a].values
y_pred=clf.predict(test)
df=pd.DataFrame({'predicted':y_pred})
df

,predicted
0,3
1,2
2,2
3,1
4,3
5,2
6,1
7,3
8,2
9,2


In [ ]:
"""params = {'class_weight':[{0:neg_weight, 1:1} for neg_weight in np.arange(1.0, 5.0, 0.5)]}
gs = GridSearchCV(estimator=clf, param_grid=params, cv=5)
gs.fit(X_train, y_train)""

In [5]:
feature_importances = [(feature, importance) for feature,importance in zip(a, clf.feature_importances_)]
feature_importances = sorted(feature_importances, key = lambda x: x[1], reverse = True)
[print('Variable: {:20} Importance: {}'.format(*pair)) for pair in feature_importances]

Variable: geo_level_3_id       Importance: 0.15009861086903142
Variable: geo_level_1_id       Importance: 0.13910401097857847
Variable: geo_level_2_id       Importance: 0.1293758313013962
Variable: age                  Importance: 0.12570366286617907
Variable: area_percentage      Importance: 0.11478220671258457
Variable: height_percentage    Importance: 0.054906705987544456
Variable: count_families       Importance: 0.025899013616322294
Variable: roof_type            Importance: 0.025729925521321865
Variable: foundation_type      Importance: 0.024530678206297132
Variable: position             Importance: 0.022233582236908092
Variable: land_surface_condition Importance: 0.01913102436595971
Variable: other_floor_type     Importance: 0.018319504100467347
Variable: count_floors_pre_eq  Importance: 0.01801437877413009
Variable: has_superstructure_mud_mortar_stone Importance: 0.01718901340507933
Variable: ground_floor_type    Importance: 0.016052003159796723
Variable: has_superstructure_tim

[None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None]

In [6]:
from sklearn.feature_selection import SelectFromModel
from sklearn.metrics import accuracy_score
sfm = SelectFromModel(clf, threshold=0.00024846532679632954)
sfm.fit(x, y)

SelectFromModel(estimator=RandomForestClassifier(bootstrap=True, class_weight=None, criterion='gini',
            max_depth=None, max_features=8, max_leaf_nodes=None,
            min_impurity_decrease=0.0, min_impurity_split=None,
            min_samples_leaf=1, min_samples_split=2,
            min_weight_fraction_leaf=0.0, n_estimators=50, n_jobs=None,
            oob_score=False, random_state=None, verbose=0,
            warm_start=False),
        max_features=None, norm_order=1, prefit=False,
        threshold=0.00024846532679632954)

In [8]:
x_important_train = sfm.transform(x)
x_important_test = sfm.transform(test)
clf_important = RandomForestClassifier(n_estimators=50, max_features=25,random_state=0, n_jobs=-1)
clf_important.fit(x_important_train, y)

RandomForestClassifier(bootstrap=True, class_weight=None, criterion='gini',
            max_depth=None, max_features=25, max_leaf_nodes=None,
            min_impurity_decrease=0.0, min_impurity_split=None,
            min_samples_leaf=1, min_samples_split=2,
            min_weight_fraction_leaf=0.0, n_estimators=50, n_jobs=-1,
            oob_score=False, random_state=0, verbose=0, warm_start=False)

In [27]:
y_pred1=clf_important.predict(x_important_test)
df1=pd.DataFrame({'damage_grade':y_pred1})
df1

,damage_grade
0,3
1,2
2,3
3,1
4,3
5,3
6,1
7,3
8,2
9,2


In [28]:
traindata = pd.read_csv("test_values.csv",sep=',')
building_id = traindata['building_id'].values
building_id

array([ 300051,   99355,  890251, ..., 1049160,  442785,  501372],
      dtype=int64)

In [29]:
df1['building_id'] = building_id
df1

,damage_grade,building_id
0,3,300051
1,2,99355
2,3,890251
3,1,745817
4,3,421793
5,3,871976
6,1,691228
7,3,896100
8,2,343471
9,2,766647


In [30]:
cols = df1.columns.tolist()
cols = cols[-1:] + cols[:-1]
cols

['building_id', 'damage_grade']

In [31]:
df1 = df1[cols]
df1

,building_id,damage_grade
0,300051,3
1,99355,2
2,890251,3
3,745817,1
4,421793,3
5,871976,3
6,691228,1
7,896100,3
8,343471,2
9,766647,2


In [33]:
export_csv = df1.to_csv (r'C:\Users\HP\Desktop\1.csv', index = None, header=True)